In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from numpy import concatenate
from math import sqrt
import random

/Users/alexellard/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('/Users/alexellard/Downloads/scenarios_data.csv', header=0, index_col=0)
df

,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,node_8_demand,node_9_demand,...,link_23_flow,link_24_flow,link_25_flow,link_26_flow,link_27_flow,link_28_flow,link_29_flow,link_30_flow,link_31_flow,link_32_flow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-3348.0,154.8,169.2,18.0,118.8,169.2,234.0,86.4,82.8,104.4,...,439.2,-205.2,-75.6,-18.0,118.8,68.4,7.2,-54.0,75.6,212.4
2017-01-01 00:30:00,-2959.2,136.8,147.6,18.0,108.0,144.0,237.6,79.2,75.6,90.0,...,392.4,-187.2,-72.0,-21.6,108.0,61.2,3.6,-54.0,72.0,183.6
2017-01-01 01:00:00,-2692.8,129.6,126.0,14.4,104.4,133.2,205.2,72.0,72.0,90.0,...,349.2,-176.4,-64.8,-18.0,93.6,50.4,3.6,-39.6,54.0,151.2
2017-01-01 01:30:00,-2379.6,104.4,100.8,10.8,86.4,122.4,180.0,68.4,57.6,82.8,...,316.8,-158.4,-54.0,-10.8,79.2,46.8,7.2,-32.4,50.4,140.4
2017-01-01 02:00:00,-2228.4,104.4,104.4,10.8,79.2,122.4,183.6,68.4,54.0,75.6,...,277.2,-144.0,-57.6,-18.0,72.0,39.6,7.2,-32.4,43.2,118.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 21:30:00,-5306.4,205.2,208.8,39.6,183.6,284.4,316.8,176.4,133.2,129.6,...,604.8,-259.2,32.4,115.2,169.2,100.8,10.8,-100.8,122.4,309.6
2017-12-31 22:00:00,-4910.4,208.8,183.6,39.6,154.8,252.0,295.2,165.6,133.2,133.2,...,565.2,-241.2,21.6,100.8,154.8,86.4,18.0,-75.6,100.8,288.0
2017-12-31 22:30:00,-4683.6,187.2,183.6,36.0,147.6,230.4,320.4,147.6,118.8,126.0,...,529.2,-226.8,21.6,100.8,147.6,86.4,14.4,-86.4,108.0,266.4


In [26]:
demands = ['node_1_demand', 'node_16_demand', 'node_29_demand']
pressures = ['node_1_pressure', 'node_16_pressure', 'node_29_pressure']
flows = ['link_1_flow', 'link_16_flow', 'link_29_flow']

parameters = [demands, pressures, flows]

In [21]:
for node in demands:
    # split into train and test sets
    train, test = train_test_split(df, test_size=0.2, random_state=0)

    # split into input and outputs
    train_X, train_y = train.drop(node, axis=1), train[node]
    test_X, test_y = test.drop(node, axis=1), test[node]

    #define model
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.9,
        colsample_bytree=0.8,
        random_state=42
    )

    # fit the model on the training data
    model.fit(train_X, train_y)

    # make predictions on the test data
    y_pred = model.predict(test_X)

    # calculate the mean squared error
    mae = mean_absolute_error(test_y, y_pred)
    print('Mean Absolute Error:', mae)
    rmse = sqrt(mean_squared_error(test_y, y_pred))
    print('Test RMSE: %.3f' % rmse)
    r2 = r2_score(test_y, y_pred)
    print('Test R^2: %.3f' % r2)

Mean Absolute Error: 10.057374208789982
Test RMSE: 13.079
Test R^2: 0.977
Mean Absolute Error: 2.290000517248563
Test RMSE: 3.040
Test R^2: 0.998
Mean Absolute Error: 1.7014532531015405
Test RMSE: 2.228
Test R^2: 0.996


In [23]:
for node in pressures:
    # split into train and test sets
    train, test = train_test_split(df, test_size=0.2, random_state=0)

    # split into input and outputs
    train_X, train_y = train.drop(node, axis=1), train[node]
    test_X, test_y = test.drop(node, axis=1), test[node]

    #define model
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.9,
        colsample_bytree=0.8,
        random_state=42
    )

    # fit the model on the training data
    model.fit(train_X, train_y)

    # make predictions on the test data
    y_pred = model.predict(test_X)

    # calculate the mean squared error
    mae = mean_absolute_error(test_y, y_pred)
    print('Mean Absolute Error:', mae)
    rmse = sqrt(mean_squared_error(test_y, y_pred))
    print('Test RMSE: %.3f' % rmse)
    r2 = r2_score(test_y, y_pred)
    print('Test R^2: %.3f' % r2)

Mean Absolute Error: 0.005532106697395996
Test RMSE: 0.008
Test R^2: 0.998
Mean Absolute Error: 0.04234998454612137
Test RMSE: 0.056
Test R^2: 1.000
Mean Absolute Error: 0.025202936891878035
Test RMSE: 0.034
Test R^2: 1.000


In [24]:
for node in flows:
    # split into train and test sets
    train, test = train_test_split(df, test_size=0.2, random_state=0)

    # split into input and outputs
    train_X, train_y = train.drop(node, axis=1), train[node]
    test_X, test_y = test.drop(node, axis=1), test[node]

    #define model
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.9,
        colsample_bytree=0.8,
        random_state=42
    )

    # fit the model on the training data
    model.fit(train_X, train_y)

    # make predictions on the test data
    y_pred = model.predict(test_X)

    # calculate the mean squared error
    mae = mean_absolute_error(test_y, y_pred)
    print('Mean Absolute Error:', mae)
    rmse = sqrt(mean_squared_error(test_y, y_pred))
    print('Test RMSE: %.3f' % rmse)
    r2 = r2_score(test_y, y_pred)
    print('Test R^2: %.3f' % r2)

Mean Absolute Error: 5.467161509892703
Test RMSE: 7.211
Test R^2: 1.000
Mean Absolute Error: 2.092597694222786
Test RMSE: 2.676
Test R^2: 0.999
Mean Absolute Error: 1.3185284382982614
Test RMSE: 1.700
Test R^2: 0.995
